In [20]:
import itertools
import pandas as pd
import datetime

def backtest_mutual_funds(filename, start='2019-01-01', amount=1000000,
                           lookback_years=1, num_funds=5):

    df = pd.read_csv(filename, parse_dates=['date'])
    df = df.copy()

    transactions = []
    portfolio = {}
    annual = []

    start_dt = pd.to_datetime(start)

    for rebalance_dt in pd.date_range(start_dt, df.date.max(), freq='12MS'):

        past_dt = rebalance_dt - pd.DateOffset(months=lookback_years)

        past_data = df[(df.date >= past_dt) & (df.date < rebalance_dt)]

        fund_returns = past_data.groupby('Scheme Code')['nav'].apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1) * 100)

        top_funds = fund_returns.sort_values(ascending=False).head(num_funds).index

        total = amount if not portfolio else sum([qty * df[(df.date >= rebalance_dt) & (df['Scheme Code'] == fund)]['nav'].values[0] for fund, qty in portfolio.items()])

        portfolio = {}

        for fund in top_funds:
            price = df[(df.date >= rebalance_dt) & (df['Scheme Code'] == fund)]['nav'].values[0]
            qty = (total / num_funds) / price

            portfolio[fund] = qty

            transactions.append({
                'date': rebalance_dt,
                'type': 'open',
                'fund': fund,
                'qty': qty,
                'price': price
            })

        annual.append({
            'date': rebalance_dt,
            'portfolio': total
        })

    CAGR = (annual[-1]['portfolio'] / annual[0]['portfolio']) ** (1 / (len(annual) / 1)) - 1

    return transactions, portfolio, annual, CAGR

def run_multiple_backtests(filename, start='2019-01-01', amount=1000000,
                           lookback_years_vals=[1, 2, 3, 4, 5], num_funds_vals=[1, 2, 3, 4, 5] ):

    results = []

    for lookback, num_funds in itertools.product(lookback_years_vals, num_funds_vals):

        transactions, portfolio, annual, CAGR = backtest_mutual_funds(filename,
                                      start=start,
                                      amount=amount,
                                      lookback_years=lookback,
                                      num_funds=num_funds)

        total_return = 100 * (annual[-1]['portfolio'] / annual[0]['portfolio'] - 1)

        results.append({
            'lookback_years': lookback,
            'num_funds': num_funds,
            'CAGR (%)': CAGR * 100,
            'total_return (%)': total_return
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv("strategy_report_1.csv", index=False)
    print(results_df)

    return results_df

if __name__ =="__main__" :

    strategies = run_multiple_backtests("mf_data.csv",
                                   start='2019-01-01', amount=1000000,
                                   lookback_years_vals=[6, 12, 24, 36, 48, 60,120], num_funds_vals=[1, 2, 3, 4, 5])

    print("All backtests complete. Summary of strategies has been saved to strategy_report.csv")


    lookback_years  num_funds   CAGR (%)  total_return (%)
0                6          1  17.556132        210.253731
1                6          2  16.523164        191.666043
2                6          3  18.411735        226.409814
3                6          4  18.242373        223.155794
4                6          5  17.532339        209.814447
5               12          1  23.294357        333.114187
6               12          2  21.652263        294.313224
7               12          3  19.916437        256.575106
8               12          4  19.331010        244.566626
9               12          5  19.884095        255.902447
10              24          1  20.254292        263.667173
11              24          2  20.996348        279.669600
12              24          3  19.177950        241.484793
13              24          4  19.472447        247.435592
14              24          5  18.982380        237.581439
15              36          1  17.323517        205.9817

In [ ]:
def